In [53]:
import torch
%cd /Users/gw/repos/prob_prog/hw/hw2/CS532-HW2/
from tests import is_tol

/Users/gw/repos/prob_prog/hw/hw2/CS532-HW2


In [35]:
two_arg_primitives = {
    '+':torch.add,
    '-':torch.subtract,
    '*':torch.multiply,
    '/':torch.divide
    }
one_arg_primitives = {
    'sqrt':torch.sqrt
}

In [4]:
from daphne import daphne
from tests import is_tol, run_prob_test,load_truth


In [66]:
%cd /Users/gw/repos/daphne
for i in range(1,14):
    fname = '../prob_prog/hw/hw2/CS532-HW2/programs/tests/deterministic/test_{}.daphne'.format(i)
    ast = daphne(['desugar', '-i', fname])
    with open(fname.replace('.daphne','.json'), 'w') as f:
        json.dump(ast, f)

/Users/gw/repos/daphne


In [71]:
def base_case(e,s,l):
    if e[0] in two_arg_primitives.keys():
        e0,arg1,arg2 = e
        E = two_arg_primitives[e0](make_tensor(arg1),make_tensor(arg2))
    elif e[0] in one_arg_primitives.keys():
        e0,arg1 = e
        E = one_arg_primitives[e0](make_tensor(arg1))
    return E,s

In [44]:
def make_tensor(v):
    return torch.tensor(v)

In [64]:
is_tol(base_case(e,1,1)[0],1.4142136)


True

In [85]:
with open('/Users/gw/repos/prob_prog/hw/hw2/CS532-HW2/programs/tests/deterministic/test_1.json') as f:
  ast = json.load(f)

In [72]:
e=data[0]
s,l=1,1
base_case(e,s,l)

(tensor(7), 1)

In [90]:
def evaluate_program(ast):
    """Evaluate a program as desugared by daphne, generate a sample from the prior
    Args:
        ast: json FOPPL program
    Returns: sample from the prior of ast
    """

    # constants
    s,l = {},{}
    c = set(one_arg_primitives.keys()).union(set(two_arg_primitives.keys()))
    e = ast[0]
    
    if e[0] in c:
        E,s = base_case(e,s,l)
        return E,s

    return None

In [92]:
ret, sig = evaluate_program(ast)

In [103]:
def run_deterministic_tests():
    
    for i in [1,2]:
        #note: this path should be with respect to the daphne path!
        ast = daphne(['desugar', '-i', '../prob_prog/hw/hw2/CS532-HW2/programs/tests/deterministic/test_{}.daphne'.format(i)])
        truth = load_truth('../prob_prog/hw/hw2/CS532-HW2/programs/tests/deterministic/test_{}.truth'.format(i))
        ret, sig = evaluate_program(ast)
        try:
            assert(is_tol(ret, truth))
        except AssertionError:
            raise AssertionError('return value {} is not equal to truth {} for exp {}'.format(ret,truth,ast))
        
        print('Test %i passed'%i)
        
    print('All deterministic tests passed')


In [104]:
%cd /Users/gw/repos/daphne
run_deterministic_tests()

/Users/gw/repos/daphne
Test 1 passed
Test 2 passed
All deterministic tests passed


In [100]:
%pwd
%ls programs
#/tests/deterministic/test_1.truth

arithmetic_circuit.daphne  homework2/
convolution.daphne
